In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

#  STEP 1: Dataset Paths 
train_dir = r"C:\Users\shivain\Downloads\Face Mask Dataset\Train"
val_dir   = r"C:\Users\shivain\Downloads\Face Mask Dataset\Validation"
test_dir  = r"C:\Users\shivain\Downloads\Face Mask Dataset\Test"

# STEP 2: Preprocess Images 
img_size = 100
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

#  STEP 3: CNN Model 
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output: Mask / No Mask
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#  STEP 4: Train Model 
print("✅ Training started...")
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

# Save the trained model
model_path = r"C:\Users\shivain\Downloads\Face Mask Dataset\mask_detector_model.h5"
model.save(model_path)
print(f"✅ Model saved at {model_path}")

# STEP 5: Real-time Detection 
print("✅ Loading model for real-time detection...")
model = load_model(model_path)
labels = ["No Mask", "Mask"]

# Load Haarcascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open Webcam
cap = cv2.VideoCapture(0)
print("✅ Real-time detection started. Press 'q' to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 4)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (img_size, img_size)) / 255.0
        reshaped = np.reshape(resized, (1, img_size, img_size, 3))
        result = model.predict(reshaped, verbose=0)
        
        # Mask / No Mask
        label = labels[int(result[0][0] > 0.5)]
        color = (0,255,0) if label == "Mask" else (0,0,255)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


D:\anaconda\envs\mask_env\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


Found 384 images belonging to 2 classes.
Found 363 images belonging to 2 classes.
Found 423 images belonging to 2 classes.
✅ Training started...
Epoch 1/5
12/12 [==============================] - 8s 621ms/step - loss: 0.8142 - accuracy: 0.6276 - val_loss: 0.2261 - val_accuracy: 0.9559
Epoch 2/5
12/12 [==============================] - 6s 523ms/step - loss: 0.1650 - accuracy: 0.9505 - val_loss: 0.1809 - val_accuracy: 0.9394
Epoch 3/5
12/12 [==============================] - 7s 568ms/step - loss: 0.1589 - accuracy: 0.9349 - val_loss: 0.2322 - val_accuracy: 0.8898
Epoch 4/5
12/12 [==============================] - 7s 568ms/step - loss: 0.1244 - accuracy: 0.9661 - val_loss: 0.0459 - val_accuracy: 0.9890
Epoch 5/5
12/12 [==============================] - 7s 562ms/step - loss: 0.0460 - accuracy: 0.9922 - val_loss: 0.0302 - val_accuracy: 0.9917
✅ Model saved at C:\Users\shivain\Downloads\Face Mask Dataset\mask_detector_model.h5
✅ Loading model for real-time detection...
✅ Real-time detection 